In [75]:
import random
import numpy as np
from scipy.stats import norm, multivariate_normal

In [76]:
# def calculate_expected_stochastic_flexibility(lagrange_multipliers, b_theta, b_zero, mean_theta, var_covar_theta):
#     mean_psi = list()
#     temp = list()
#     sfindex_list = list()
# 
#     for i in lagrange_multipliers:
#         mu_psi = i @ (b_theta @ mean_theta + b_zero)
#         var_psi = (i @ b_theta) @ var_covar_theta @ (i @ b_theta).T
#         stdev_psi = np.sqrt(var_psi)
#         z = -mu_psi/stdev_psi
#         delta = norm.cdf(z)
#         mean_psi.append(mu_psi)
#         temp.append((i @ b_theta).tolist())
#         sfindex_list.append(round(delta,4))
# 
#     var_covar_psi = np.array(temp) @ var_covar_theta @ np.transpose(np.array(temp))    
#     indices = [i for i, val in enumerate(sfindex_list) if val == 1]
# 
#     mean_psi_np = np.delete(np.array(mean_psi), indices, axis=0)
#     var_covar_psi = np.delete(var_covar_psi, indices, axis=0)
#     var_covar_psi = np.delete(var_covar_psi, indices, axis=1)
# 
# 
#     multivariate_dist = multivariate_normal(mean_psi_np, var_covar_psi, allow_singular=True)
# 
#     esf_index = multivariate_dist.cdf([0]*len(mean_psi_np))
# 
#     # return np.array(mean_psi), var_covar_psi, sfindex_list
# 
#     return esf_index

In [77]:
def calculate_stochastic_flexibility(mean_theta, var_covar_theta,
                                              lagrange_multipliers=None, b_theta=None, b_zero=None,
                                              psi_b_theta=None, psi_b_zero=None):
    # Check feasibility function inputs
    if psi_b_theta is None:
        if lagrange_multipliers is None or b_theta is None:
            raise ValueError("Provide either 'psi_b_theta' directly or both 'lagrange_multipliers' and 'b_theta'.")
        psi_b_theta = lagrange_multipliers @ b_theta

    if psi_b_zero is None:
        if lagrange_multipliers is None or b_zero is None:
            raise ValueError("Provide either 'psi_b_zero' directly or both 'lagrange_multipliers' and 'b_zero'.")
        psi_b_zero = lagrange_multipliers @ b_zero

    # Compute mean and std dev of feasibility functions
    mean_psi = psi_b_theta @ mean_theta + psi_b_zero
    stdev_psi = np.sqrt(np.einsum('ij,jk,ik->i', psi_b_theta, var_covar_theta, psi_b_theta))

    # SF index values
    z = -mean_psi / stdev_psi
    sfindex_array = np.round(norm.cdf(z), 4)
    print(sfindex_array)
    # Remove rows/cols where SF index == 1
    keep = sfindex_array != 1
    mean_psi_filtered = mean_psi[keep]
    psi_b_theta_filtered = psi_b_theta[keep]

    var_covar_psi = psi_b_theta_filtered @ var_covar_theta @ psi_b_theta_filtered.T

    # Multivariate CDF
    sf_index = multivariate_normal(mean_psi_filtered, var_covar_psi, allow_singular=True).cdf(np.zeros_like(mean_psi_filtered))

    return sf_index

In [78]:
# Data for Example 1 in Pistikopoulos and Mazzuchi 1990
# 
# lagm = np.array([
#     [0.5, 0.5, 0, 0, 0],
#     [0.5, 0, 0.5, 0, 0],
#     [0.5, 0, 0, 0.5, 0],
#     [0, 0.5, 0, 0, 0.5],
#     [0, 0, 0.5, 0, 0.5],
#     [0, 0, 0, 0.5, 0.5]
# ])
# 
# b_j_theta = np.array([
#     [0, 1.5, 0 ,0],
#     [-1.5, -2, -2, 0],
#     [-1.5, -2, -1, 0],
#     [-1.5, -2, -1, -2],
#     [1.5, 2, 1, 3]
# ])
# 
# b_j_zero = np.array([-525, 2777, 2044, 2830, -3153])
mean_theta_1 = np.array([620, 388, 583, 313])
var_covar_theta_1 = np.diag([69.39, 69.39, 69.39, 69.39])

psi_b_theta_1 = np.array([
    [-0.75, -0.25, -1, 0],
    [-0.75, -0.25, -0.5, 0],
    [-0.75, -0.25, -0.5, -1],
    [0, 0, -0.5, 1.5],
    [0, 0, 0, 1.5],
    [0, 0, 0, 0.5]
])

psi_b_zero_1 = np.array([1126, 759.5, 1152.5, -188, -554.5, -161.5])
sf = calculate_stochastic_flexibility(mean_theta=mean_theta_1, var_covar_theta=var_covar_theta_1, psi_b_theta=psi_b_theta_1, psi_b_zero=psi_b_zero_1)
sf

[0.9632 1.     0.8902 0.7761 1.     0.885 ]


0.6632761366386204

In [79]:
# Data for Example 2 in Pistikopoulos and Mazzuchi 1990

psi_b_theta_2 = np.array([
    [0, 0, 0.8, 0, 0, 1, 0.8],
    [0, 0, 1, 0, 0, 0, 1],
    [5/18, 2.5/18, 1/3, 0, 12.5/36, 1, 1/3],
    [4, 2, 2, -3.5, 5, 6, 2],
    [0, 0, -2, -2.5, 0, 0, -2],
    [0, 0, 0, 0, 0, 1, 0]
])

psi_b_zero_2 = np.array([-858, -700, -4907.5/6, -6045, 2145, -325])

mean_theta_2 = np.array([400, 450, 400, 430, 310, 290, 295])
var_covar_theta_2 = np.diag([69.39, 69.39, 69.39, 69.39, 69.39, 69.39, 69.39])

sf = calculate_stochastic_flexibility(mean_theta=mean_theta_2, var_covar_theta=var_covar_theta_2, psi_b_theta=psi_b_theta_2, psi_b_zero=psi_b_zero_2)

[0.83   0.6644 0.9333 1.     1.     1.    ]


In [80]:
sf

0.6262547915607823